In [34]:
from pandas import read_excel
from pandas import DataFrame
from pandas import concat
from pandas import Series
import numpy
from numpy import append
from numpy import array
from numpy import diff
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

In [35]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return array(diff)

In [36]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [37]:
# Get technical indicators
def get_technical_indicators(data):
    # Create Moving Average
    data = DataFrame(data)
    dataset = DataFrame()
    dataset['ma5'] = data.rolling(window=5).mean()
    dataset['ma20'] = data.rolling(window=25).mean()
    
    # Create MACD
    dataset['25ema'] = data.ewm(span=25).mean()
    dataset['10ema'] = data.ewm(span=10).mean()
    dataset['MACD'] = dataset['10ema']-dataset['25ema']

    # Create Bollinger Bands
    dataset['20sd'] = data.rolling(window = 20).std()
    dataset['upper_band'] = dataset['ma20'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma20'] - (dataset['20sd']*2)
    
    # Create Exponential moving average
    dataset['ema'] = data.ewm(com=0.5).mean()
    
    # Create Momentum
    dataset['momentum'] = data-1
    # dataset['log_momentum'] = log(dataset['momentum'])
    return dataset.dropna()

In [38]:
def timeseries_to_supervised(data, lag):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    return DataFrame(df)

In [39]:
# fit ML model
def model_fit(data):
    X, y = data.iloc[:,:-1], data.iloc[:,-1]
    param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001],
                  'kernel': ['rbf', 'poly', 'sigmoid']}
    model = RandomizedSearchCV(SVR(),param_grid,cv=5,n_iter=20,refit="AUC",verbose=1,n_jobs=-1)
    model_fit = model.fit(X,y)
    return model_fit

In [40]:
# make a one-step forecast
def model_predict(model, X):
    yhat = model.predict(X.reshape(1, -1))
    return yhat[0]

In [41]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X):
    inverted = scaler.inverse_transform(array(X).reshape(-1,1))
    return array(inverted)

In [42]:
# invert differenced value
def invert_difference(last_ob, yhat_diff, interval=1):
    return numpy.r_[last_ob, yhat_diff].cumsum()

In [43]:
# run a repeated experiment
def experiment(series, features):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values)
    
    # split data into train and test-sets
    len_test = 150
    train = diff_values[:-len_test]
    test = diff_values[-len_test:]
    
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    
    # get technical indicators
    df = get_technical_indicators(train_scaled)
    
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(train_scaled, features)
    supervised_values = supervised.values[features:,:]
    
    data = concat([df, DataFrame(supervised_values)],axis=1).dropna()
    
    # fit the base model
    mod = model_fit(data)

    # predict first row
    testX, testy = data.iloc[-1, :-1], data.iloc[-1, -1]

    # forecast test dataset
    predictions = list()
    for i in range(len(test)):
        # predict
        yhat_scaled = model_predict(mod, array(testX))
        # store forecast
        train_scaled = append(train_scaled,yhat_scaled)
        df = get_technical_indicators(train_scaled)
        supervised = timeseries_to_supervised(train_scaled, features)
        supervised_values = supervised.values[features:,:]
        data = concat([df, DataFrame(supervised_values)],axis=1)
        data = DataFrame(data).dropna()
        testX = data.iloc[-1,:-1]
        predictions.append(yhat_scaled)
    
    predictions = invert_scale(scaler,predictions)
    predictions = invert_difference(raw_values[-len_test],predictions[:,0])
    
    return DataFrame(predictions), DataFrame(raw_values[-len_test:])

In [44]:
path = "C:\\Users\\fatim\\Box\\Habbab Fatima\\Experiment set 3\\DataSets\\"
asset_list = read_excel("C:\\Users\\fatim\\Box\\Habbab Fatima\\Experiment set 6\\asset_list.xlsx")

no_assets = len(asset_list)

predictions = DataFrame()
test_data = DataFrame()

for i in range(0,no_assets):
    ticker = asset_list.iloc[i,0]
    data = read_excel(path+ticker+".xlsx")
    series = data[["Close"]].dropna()
    
    no_features = 2
    
    yhat, test = experiment(series, no_features)

    predictions[ticker] = yhat
    test_data[ticker] = test

In [45]:
test_data.to_excel("test150.xlsx")